# Paraphrase Classification by Fine-tuning BERT

Pre-trained language
representations have been shown to improve many downstream NLP tasks such as
question answering, and natural language inference. To apply pre-trained
representations to these tasks, there are two strategies:

1. **feature-based**
approach, which uses the pre-trained representations as additional
features to
the downstream task.
2. **fine-tuning** based approach, which trains the
downstream tasks by
fine-tuning pre-trained parameters.

While feature-based
approaches such as ELMo [3] (introduced in the previous tutorial) are effective
in improving many downstream tasks, they require task-specific architectures.
Devlin, Jacob, et al proposed BERT [1] (Bidirectional Encoder Representations
from Transformers), which **fine-tunes** deep bidirectional representations on a
wide range of tasks with minimal task-specific parameters, and obtained state-
of-the-art results.

In this tutorial, we will focus on fine-tuning with the
pre-trained BERT model to classify semantically equivalent sentence pairs.
Specifically, we will:

1. load the state-of-the-art pre-trained BERT model and
attach an additional layer for classification,
2. process and transform sentence
pair data for the task at hand, and
3. fine-tune BERT model for sentence
classification.

## Preparation

To run this tutorial locally, please:

- [install gluonnlp](http://gluon-nlp.mxnet.io/#installation), and
- click the
download button at the top of the tutorial page to get all related code.

Then
we start with some usual preparation such as importing libraries
and setting the
environment.

### Load MXNet and GluonNLP

In [1]:
import warnings
warnings.filterwarnings('ignore')

import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from bert import data, model

### Set Environment

In [2]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.gpu(0)

## Use the Pre-trained BERT Model

The list of pre-trained BERT model available
in GluonNLP can be found
[here](../../model_zoo/bert/index.rst).

In this
tutorial, we will load the BERT
BASE model trained on uncased book corpus and
English Wikipedia dataset in
GluonNLP model zoo.

### Get BERT

Let's first take
a look at the BERT model
architecture for sentence pair classification below:
<div style="width:
500px;">![bert-sentence-pair](bert-sentence-pair.png)</div>
where the model takes a pair of
sequences and *pools* the representation of the
first token in the sequence.
Note that the original BERT model was trained for
masked language model and next
sentence prediction tasks, which includes layers
for language model decoding and
classification. These layers will not be used
for fine-tuning sentence pair classification.

Let's load the
pre-trained BERT
using the model API in GluonNLP, which returns the vocabulary
along with the
model. We include the pooler layer of the pre-trained model by setting
`use_pooler` to `True`.

In [3]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12', 
                                             dataset_name='book_corpus_wiki_en_uncased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)
print(bert_base)

BERTModel(
  (encoder): BERTEncoder(
    (dropout_layer): Dropout(p = 0.1, axes=())
    (layer_norm): BERTLayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
    (transformer_cells): HybridSequential(
      (0): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): MultiHeadAttentionCell(
          (_base_cell): DotProductAttentionCell(
            (_dropout_layer): Dropout(p = 0.1, axes=())
          )
          (proj_query): Dense(768 -> 768, linear)
          (proj_key): Dense(768 -> 768, linear)
          (proj_value): Dense(768 -> 768, linear)
        )
        (proj): Dense(768 -> 768, linear)
        (ffn): BERTPositionwiseFFN(
          (ffn_1): Dense(768 -> 3072, linear)
          (activation): GELU()
          (ffn_2): Dense(3072 -> 768, linear)
          (dropout_layer): Dropout(p = 0.1, axes=())
          (layer_norm): BERTLayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
        )
        (la

### Model Definition for Sentence Pair Classification

Now that we have loaded
the BERT model, we only need to attach an additional layer for classification.
The `BERTClassifier` class uses a BERT base model to encode sentence
representation, followed by a `nn.Dense` layer for classification.

In [4]:
model = model.classification.BERTClassifier(bert_base, num_classes=2, dropout=0.1)
# only need to initialize the classifier layer.
model.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
model.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = mx.gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

## Data Preprocessing for BERT


### Dataset

For demonstration purpose, we use
the dev set of the
Microsoft Research Paraphrase Corpus dataset. The file is
named 'dev.tsv'. Let's take a look at the first few lines of the raw dataset.

In [5]:
tsv_file = io.open('dev.tsv', encoding='utf-8')
for i in range(5):
    print(tsv_file.readline())

﻿Quality	#1 ID	#2 ID	#1 String	#2 String

1	1355540	1355592	He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .	" The foodservice pie business does not fit our long-term growth strategy .

0	2029631	2029565	Magnarelli said Racicot hated the Iraqi regime and looked forward to using his long years of training in the war .	His wife said he was " 100 percent behind George Bush " and looked forward to using his years of training in the war .

0	487993	487952	The dollar was at 116.92 yen against the yen , flat on the session , and at 1.2891 against the Swiss franc , also flat .	The dollar was at 116.78 yen JPY = , virtually flat on the session , and at 1.2871 against the Swiss franc CHF = , down 0.1 percent .

1	1989515	1989458	The AFL-CIO is waiting until October to decide if it will endorse a candidate .	The AFL-CIO announced Wednesday that it will decide in October whether to endorse a candidate before the primaries .



The file contains 5 columns, separated by tabs.
The first line of
the file explains each of these columns:
0. the label indicating whether the two
sentences are semantically equivalent
1. the id of the first sentence in this
sample
2. the id of the second sentence in this sample
3. the content of the
first sentence
4. the content of the second sentence

For our task, we are
interested in the 0th, 3rd and 4th columns.
To load this dataset, we can use the
`TSVDataset` API:

In [6]:
# skip the first line, which is the schema
num_discard_samples = 1
# split fields by tabs
field_separator = nlp.data.Splitter('\t')
# fields to select from the file
field_indices = [3, 4, 0]
data_train_raw = nlp.data.TSVDataset(filename='dev.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples,
                                 field_indices=field_indices)
sample_id = 0
# sentence a
print(data_train_raw[sample_id][0])
# sentence b
print(data_train_raw[sample_id][1])
# 1 means equivalent, 0 means not equivalent
print(data_train_raw[sample_id][2])

He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .
" The foodservice pie business does not fit our long-term growth strategy .
1


To use the pre-trained BERT model, we need to preprocess the data in the same
way it was trained. The following figure shows the input representation in BERT:
<div style="width: 500px;">![bert-embed](bert-embed.png)</div>

We will use
`BERTDatasetTransform` to perform the following transformations:
- tokenize
the
input sequences
- insert [CLS] at the beginning
- insert [SEP] between sentence
one and sentence two, and at the end
- generate segment ids to indicate whether
a token belongs to the first sequence or the second sequence.
- generate valid
length

In [7]:
# use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)
# maximum sequence length
max_len = 128
# the labels for the two classes
all_labels = ["0", "1"]
# whether to transform the data as sentence pairs.
# for single sentence classification, set pair=False
# for regression task, set class_labels=None
# for inference without label available, set has_label=False
pair = True
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                                                class_labels=all_labels,
                                                has_label=True,
                                                pad=True,
                                                pair=pair)
data_train = data_train_raw.transform(transform)

print('vocabulary used for tokenization = \n%s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = \n%s'%data_train[sample_id][0])
print('valid length = \n%s'%data_train[sample_id][1])
print('segment ids = \n%s'%data_train[sample_id][2])
print('label = \n%s'%data_train[sample_id][3])

vocabulary used for tokenization = 
Vocab(size=30522, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 1
[CLS] token id = 2
[SEP] token id = 3
token ids = 
[    2  2002  2056  1996  9440  2121  7903  2063 11345  2449  2987  1005
  1056  4906  1996  2194  1005  1055  2146  1011  2744  3930  5656  1012
     3  1000  1996  9440  2121  7903  2063 11345  2449  2515  2025  4906
  2256  2146  1011  2744  3930  5656  1012     3     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1]
valid length = 
44
segment ids = 

## Fine-tune BERT Model

Putting everything together, now we can fine-tune the
model with a few epochs. For demonstration, we use a fixed learning rate and
skip validation steps.

In [8]:
batch_size = 16
lr = 5e-5
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[1]) for item in data_train],
                                            batch_size=batch_size,
                                            shuffle=True)
bert_dataloader = mx.gluon.data.DataLoader(data_train, batch_sampler=train_sampler)

trainer = mx.gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': lr})

# collect all differentiable parameters
# grad_req == 'null' indicates no gradients are calculated (e.g. constant parameters)
# the gradients for these params are clipped later
params = [p for p in model.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

log_interval = 4
num_epochs = 4
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(bert_dataloader):
        with mx.autograd.record():
            
            # load data to GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)
            
            # forward computation
            out = model(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # backward computation
        ls.backward()
        
        # gradient clipping
        trainer.allreduce_grads()
        nlp.utils.clip_grad_global_norm(params, 1)
        trainer.update(1)

        step_loss += ls.asscalar()
        metric.update([label], [out])
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0

[Epoch 0 Batch 4/31] loss=0.7040, lr=0.0000500, acc=0.542
[Epoch 0 Batch 8/31] loss=0.7539, lr=0.0000500, acc=0.604
[Epoch 0 Batch 12/31] loss=0.5322, lr=0.0000500, acc=0.638
[Epoch 0 Batch 16/31] loss=0.5775, lr=0.0000500, acc=0.653
[Epoch 0 Batch 20/31] loss=0.7571, lr=0.0000500, acc=0.646
[Epoch 0 Batch 24/31] loss=0.6011, lr=0.0000500, acc=0.658
[Epoch 0 Batch 28/31] loss=0.6654, lr=0.0000500, acc=0.647
[Epoch 1 Batch 4/31] loss=0.4305, lr=0.0000500, acc=0.740
[Epoch 1 Batch 8/31] loss=0.6095, lr=0.0000500, acc=0.723
[Epoch 1 Batch 12/31] loss=0.6919, lr=0.0000500, acc=0.677
[Epoch 1 Batch 16/31] loss=0.5523, lr=0.0000500, acc=0.696
[Epoch 1 Batch 20/31] loss=0.4112, lr=0.0000500, acc=0.700
[Epoch 1 Batch 24/31] loss=0.6507, lr=0.0000500, acc=0.684
[Epoch 1 Batch 28/31] loss=0.6448, lr=0.0000500, acc=0.683
[Epoch 2 Batch 4/31] loss=0.3918, lr=0.0000500, acc=0.791
[Epoch 2 Batch 8/31] loss=0.6558, lr=0.0000500, acc=0.705
[Epoch 2 Batch 12/31] loss=0.2092, lr=0.0000500, acc=0.731
[Ep

## Conclusion

In this tutorial, we show how to fine-tune a sentence pair
classification model with pre-trained BERT parameters. In GluonNLP, this can be
done with just a few simple steps: apply BERT-style data transformation to
preprocess the data, automatically download the pre-trained model, and feed the
transformed data into the model. For demonstration purpose, we skipped the
warmup learning rate
schedule and validation on dev dataset used in the original
implementation. Please visit the
[BERT model zoo webpage](../../model_zoo/bert/index.rst), or the scripts/bert folder
in the Github repository for complete fine-tuning scripts.

## References

[1] Devlin, Jacob, et al. "Bert:
Pre-training of deep
bidirectional transformers for language understanding."
arXiv preprint
arXiv:1810.04805 (2018).

[2] Dolan, William B., and Chris
Brockett.
"Automatically constructing a corpus of sentential paraphrases."
Proceedings of
the Third International Workshop on Paraphrasing (IWP2005). 2005.
[3] Peters,
Matthew E., et al. "Deep contextualized word representations." arXiv
preprint
arXiv:1802.05365 (2018).